# Lab 3: Extending Logistic Regression

Author: Ephraim Sun (Masters), Jadon Swearingen (Undergrad), Adeeb Abdul Taher (Masters)


In [ ]:
pip install scikit-learn termcolor plotly imblearn squarify pandas matplotlib seabor matplotlib-ven tqd missingno

Here, we import most of the librarys useful for this sort of data analysis. We looked at the file that came with the data set to look at potential imports.

In [ ]:
# Import data processing & analysis packages.

## Data Processing.
import pandas as pd
import numpy as np
import os

## Data Visualisation.
import matplotlib.pyplot as plt
import seaborn as sns
from termcolor import colored
import plotly.express as px
from matplotlib_venn import venn2

## Machine Learning.

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,accuracy_score,classification_report
from tqdm import tqdm
from itertools import cycle
from sklearn import metrics
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import feature_selection
import squarify
# import imblearn

## Warning indication.
import warnings
warnings.filterwarnings('always') 

# Preparation and Overview



The purpose of the Income Prediction DataSet provided by Zindi is to analyze and classify if a person earns more or less than \$50,000 based on various data variables. Specifically, we are given a random population of over 100,000 individuals to test on 200,000 training data. The data is collected from a random American population where the media income was about $50,000 and is provided by the company Zindi. Third parties like Zindi are interested because they will use the machine learning algorithm to classify further individuals and people, giving them a score based on these model accuracy. Furthermore, third parties can use this data in applications like Finance (credit scoring and loans, fraud detection), Healthcare (health insurance and healthcare programs), as well as other areas like HR and Public Policy. The current predicition algorithm that will be considered useful to other parties is that it must be better than 96%, which another person has achieved.

Here, we make sure that the data is read correctly, as well as see what a few rows look like and describe the data.

In [ ]:
# Import Datasets.

df = pd.read_csv("Train.csv")

# Display the first 5 rows of the train dataset.
df.head(5)

In [ ]:
df.describe()

In [ ]:
print(df.dtypes)
print('===========')
print(df.info())

Here, we start to make some initial graphs. This is adapted from the slideshow from eric larson. We look at all of the data, seeing which are null and sort it by class and age, which are two values that we want to use.

In [ ]:
# this python magics will allow plot to be embedded into the notebook
import matplotlib
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore', DeprecationWarning)
%matplotlib inline 

# plt.subplots(figsize=(20, 15))
# External package: conda install missingno 
import missingno as mn

mn.matrix(df)
plt.title("Not Sorted",fontsize=22)

plt.figure()
mn.matrix(df.sort_values(by=["class","age"]))
plt.title("Sorted",fontsize=22)
plt.show()

### Data Understanding

#### Data Types

We have a large dataset with 42 variables. Therefore, we have decided to narrow down to 10 specific data sets that our team finds valuable along wih its corresponding data variable


| Variable    | Data Type | Description | 
| -------- | ------- | ------- |
| Age  |  Integer (int64)   | The age of the individual | 
| Gender |  Categorical (object)(object)    | If they are male or female
| Education    |  Categorical (object)   | What level of education they are in: high school, children, some college but no degree, bachelors degree(BA, AB, BS), 7th and 8th grade
| Class   |  Categorical (object)  | This is type of of employment of the individual, which includes private, Self-employed-not incorportated, Local government, State government, Self-employed-incorporated, etc
| Employment Commitment    |  Categorical (object)  | This is the employment Status of the individual: Children or Amred Forces, Full-time schedules, Not in labor force, PT for non-econ reasons usually FT, Umeployed full-time, 
| Citizenship    |  Categorical (object)  | What is citizenship status in regards to the United States: Native, Foreign born - Not a citizen of US, Foreign born - US citizen by naturalization, Native - Born abroad of American Parents, Native - Born in Puerto Rice or US Outlying
| Wage per hour    |  Integer (int64)  | How much they make per hour
| Working_week_per_year   |  Integer (int64)   | How many weeks per year they work (0- 52)
| income_above_limit   |  Categorical (object)  | Is the person above or below 50,000

#### Data Quality

Entries chosen that are missing data: Class

Many of our columns has missing values. For example, "Class," which we chose to include in our analysis, has only 50% non-null values (104254 out of 209499) meaning there are 105245 values that are null. These quality issues exists because some people don't want to include their class when filling out surveys. We deal with this by eliminating all of these rows that don't have class, because 50% is too much to impute. We could have tried to use nearest neighbor imputation or split-combine-impute, but this would likely skew the results. Many other columns had similar issues, but we decided that they had little impact on the income predictor and so decided to eliminate the entire columns and to choose the 9 most likely to influence the income.

We also decided to replace string data for income_above_limit with binary 0s and 1s for better processing with the graphs. Finally, we grouped and consolidated some of the categorical data into smaller bins in order to better understand the output.

In [ ]:
# let's clean the dataset a little before moving on

# 1. Remove attributes that just arent useful for us
for col in ['ID', 'education_institue', 'marital_status', 'is_hispanic', 'unemployment_reason', 'industry_code', 'industry_code_main', 'occupation_code', 'occupation_code_main', 'total_employed', 'household_stat', 'household_summary', 'under_18_family', 'veterans_admin_questionnaire', 'tax_status','gains', 'education_institute', 'employment_stat', 'is_labor_union', 'vet_benefit', 'losses', 'stocks_status', 'mig_year', 'country_of_birth_own',  'country_of_birth_father', 'country_of_birth_mother', 'migration_code_change_in_msa', 'migration_prev_sunbelt', 'migration_code_move_within_reg', 'residence_1_year_ago', 'migration_code_change_in_reg', 'old_residence_reg', 'old_residence_state', 'importance_of_record'  ]:
    if col in df:
        del df[col]
        
# Drop class
df.dropna(inplace=True)


# change gender to male - 1, female - 0
# df['gender'].replace({0:'Female',1:'Male'},inplace=True)
# df['gender'] = np.select(
#     [df['gender'].eq(0), df['gender'].eq(1)], ['Female', 'Male'], default=np.nan
# )

df['gender'].replace((' Male', ' Female'), (1, 0), inplace=True)

# change income value to binary
df['income_above_limit'].replace(('Above limit', 'Below limit'), (1, 0), inplace=True)
# df['education'].replace(('10th grade'), ('test'), inplace=False)
df['education'].replace([' 12th grade no diploma', ' 11th grade',' 10th grade',' 9th grade',' 1st 2nd 3rd or 4th grade',' 5th or 6th grade',' Less than 1st grade',' 7th and 8th grade'], 'Less than high school', inplace=True)
df['education'].replace([' Some college but no degree'], ' High school graduate', inplace=True)
df['education'].replace([' Doctorate degree(PhD EdD)',], 'PHD', inplace=True)
df['education'].replace([' Prof school degree (MD DDS DVM LLB JD)',' Masters degree(MA MS MEng MEd MSW MBA)'], 'Master\'s/Prof', inplace=True)
df['education'].replace([' Associates degree-occup /vocational',' Associates degree-academic program'], 'Associates', inplace=True)


# print(f"the value is {df['education'].unique()}")

#('Less than high school', 'Less than high school',"Less than high school","Less than high school")


df.info()

mn.matrix(df)
plt.title("Not Sorted",fontsize=22)

plt.figure()
mn.matrix(df.sort_values(by=["class","age"]))
plt.title("Sorted",fontsize=22)
plt.show()

In [ ]:
df.head(5)

# Data Visualization

We tried to explore the wage rate trends with respect to education and class at first, but we encountered some discrepancies in the data. which led us to implore ways to correct them. For example, using Wage Rate feature led to all self-employed people having 0 dollars per hour marked down, which isn't very intuitive at first, but further investigation led us to the conclusion that when people weren't paid hourly, they simply put down a 0 which made the graphs considerably skewed.  This made us reconsider using the wage rate for the final graphs without at least first imputing data. 

In [ ]:
plt.style.use('ggplot')

fig = plt.figure(figsize=(30,5))

plt.subplot(1,3,3)


df['age'].plot.hist(bins=40)
df['age'].plot.kde(bw_method=0.1, secondary_y=True)
plt.xlim(0, 100)
plt.title('Age Distribution')

plt.show()



In [ ]:
# plot the correlation matrix 
plt.subplots(figsize=(10, 8))
vars_to_use = ['age', 'working_week_per_year', 'wage_per_hour', 'income_above_limit'] # pick vars
plt.pcolor(df[vars_to_use].corr()) # do the feature correlation plot

# fill in the indices
plt.yticks(np.arange(0.5, len(vars_to_use), 1), vars_to_use)
plt.xticks(np.arange(0.5, len(vars_to_use), 1), vars_to_use)
plt.colorbar()
plt.show()

In [ ]:
# first group the data
df_grouped = df.groupby(by=['education'])

# tabulate survival rates of each group
wage_rate = df_grouped.wage_per_hour.sum() / df_grouped.wage_per_hour.count()

# plot the bar chart using builtin pandas API
fig, ax = plt.subplots(figsize=(5, 5))  # Increase the figure size to make the plot taller
ax = wage_rate.plot(kind='barh')
ax.set_title('Wage Rate by Education')
plt.show()


df_grouped = df.groupby(by=['class'])

# tabulate survival rates of each group
wage_rate = df_grouped.wage_per_hour.sum() / df_grouped.wage_per_hour.count()

# plot the bar chart using builtin pandas API
fig, ax = plt.subplots(figsize=(5, 5))  # Increase the figure size to make the plot taller
ax = wage_rate.plot(kind='barh')
ax.set_title('Wage Rate by Class')
plt.show()


## Data Discovery


In [ ]:
# sns boxplot
plt.subplots(figsize=(30, 10))

plt.subplot(1,3,1)
sns.boxplot(x="gender", y="age", hue="income_above_limit", data=df)
plt.title('Boxplot of gender vs age and income above limit')



# plt.subplot(1,3,3)
# sns.swarmplot(x="gender", y="age", hue="income_above_limit", data=df, s=3) # s controls marker size (like bins or bw)
# plt.title('Swarm Example')

plt.show()

plt.subplots(figsize=(30, 10))
plt.subplot(1,3,2)
sns.violinplot(x="gender", y="age", hue="income_above_limit", data=df)
plt.title('Violin of gender vs age and income above limit')
plt.show()

plt.subplots(figsize=(60, 10))
plt.subplot(1,3,2)
sns.violinplot(x="education", y="age", hue="income_above_limit", data=df)
plt.title('Violin of education vs age and income above limit')
plt.show()

plt.subplots(figsize=(60, 10))
plt.subplot(1,3,2)
sns.violinplot(x="citizenship", y="age", hue="income_above_limit", data=df)
plt.title('Violin of citizenship vs age and income above limit')
plt.show()

plt.subplots(figsize=(60, 10))
plt.subplot(1,3,2)
sns.violinplot(x="employment_commitment", y="age", hue="income_above_limit", data=df)
plt.title('Violin of employment_commitment vs age and income above limit')
plt.show()

## Question 1:
How evident is the wage gap between men and women? 

[Gender Wage Violin Image]

We can notice that there are more men than women who fall below the income threshold. By observing that there are more men than women who fall below the income threshold, we are witnessing a significant disparity in income distribution between genders within the dataset. This finding highlights the existence of a potential wage gap between men and women, where women, on average, may be earning less than men.



## Question 2:
What are the trends of income with education 

[Education Mutated Violin]

It becomes pretty evident from the graph that having at least a Bachelors degree greatly increases your probablity of having an income above the median. Highschool graduates tend to be below the limit during their younger years but a few manage to cross the threshold. Higher levels of education are often associated with access to higher-paying jobs, increased skills and expertise, and greater opportunities for career advancement. This finding highlights the importance of investing in education and lifelong learning as a means of increasing economic opportunities and improving financial outcomes.


## Question 3

What are the trends of employee commitment

From the violin graph, you can notice that if you are unempoloyed, not in the labor force, or unemployed part time, than you tend to be below the limit while those who are employed full time or armed forces tend to be above the limit.


In [ ]:
sns.violinplot(x="gender", y="age", hue="income_above_limit", data=df,
               split=True, # split across violins
               inner="quart", # show innner stats like mean, IQR,
               scale="count") # scale the size of the plot by the count within each group

plt.show()

plt.subplots(figsize=(20, 10))  # Increase the figure size to make the plot wider

sns.violinplot(x="education", y="age", hue="income_above_limit", data=df,
               split=True,  # split across violins
               inner="quart",  # show inner stats like mean, IQR,
               scale="count")  # scale the size of the plot by the count within each group

plt.show()

plt.subplots(figsize=(20, 10))  # Increase the figure size to make the plot wider
sns.violinplot(x="citizenship", y="age", hue="income_above_limit", data=df,
               split=True,  # split across violins
               inner="quart",  # show inner stats like mean, IQR,
               scale="count")  # scale the size of the plot by the count within each group

plt.show()

plt.subplots(figsize=(25, 10))  # Increase the figure size to make the plot wider
sns.violinplot(x="employment_commitment", y="age", hue="income_above_limit", data=df,
               split=True,  # split across violins
               inner="quart",  # show inner stats like mean, IQR,
               scale="count")  # scale the size of the plot by the count within each group

plt.show()

In [ ]:
from plotly.graph_objs import Scatter, Layout
from plotly.graph_objs.scatter import Marker
from plotly.graph_objs.layout import XAxis, YAxis
import plotly
import numpy as np
# let's manipulate the example to serve our purposes

# limit the dataframe to 10000 values
df_limit = df.head(10000)

# plotly allows us to create JS graph elements, like a scatter object
# we put together a dictionary of plotting elements
# 'data': is the key where we place plotting elements
plotly.offline.iplot({
    'data':[
        # plot siblings and spouses versus age as scatter plot
        # and make mouseover text for if they survived
        # create markers that are sized based on the fare paid
        Scatter(x=df_limit.wage_per_hour.values+np.random.rand(*df_limit.wage_per_hour.shape)/2,
                y=df_limit.age,
                text=df_limit.income_above_limit.values.astype(str),
                marker=Marker(size=df_limit.working_week_per_year, sizemode='area', sizeref=1,),
                mode='markers')
            ],
    'layout': Layout(xaxis=XAxis(title='Wage_per_hour'),
                     yaxis=YAxis(title='Age'),
                     title='Age and Family Size (Marker Size==Fare)')
}, show_link=False)

## Exception Work (With UMAP)
We tried to explore the wage rate trends with respect to education and class at first, but we encountered some discrepancies in the data. which led us to implore ways to correct them. For example, using Wage Rate feature led to all self-employed people having 0 dollars per hour marked down, which isn't very intuitive at first, but further investigation led us to the conclusion that when people weren't paid hourly, they simply put down a 0 which made the graphs considerably skewed.  This made us reconsider using the wage rate for the final graphs without at least first imputing data. 


In [ ]:
# pip install scikit-learn numba umap-learn umap-learn[plot]


In [ ]:
# pip install pandas matplotlib datashader bokeh holoviews colorcet scikit-image


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import umap

# Assuming 'df' is your DataFrame containing the income data

# Define selected columns including 'education', 'gender', and 'wage_per_hour'
selected_columns = ["education", "gender", "wage_per_hour"]

# Clean data to only include selected columns
cleaned_data = df[selected_columns]

# Encode 'education' column to numerical values
label_encoder = LabelEncoder()
cleaned_data['education'] = label_encoder.fit_transform(cleaned_data['education'])

# Standardize the data
scaled_data = StandardScaler().fit_transform(cleaned_data)

# Reduce dimensionality using UMAP with default parameters
umap_model = umap.UMAP()
embedding = umap_model.fit_transform(scaled_data)

# Define a color mapping based on 'gender' and 'education' columns
color_mapping_combined = cleaned_data['gender'] * len(cleaned_data['education'].unique()) + cleaned_data['education']

# Define a color palette
palette = sns.color_palette("tab10", len(cleaned_data['gender'].unique()) * len(cleaned_data['education'].unique()))

# Plotting
plt.figure(figsize=(10, 6))
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=[palette[x] for x in color_mapping_combined]
)
plt.gca().set_aspect('equal', 'datalim')
plt.title('UMAP projection of the Income Data', fontsize=16)
plt.xlabel('UMAP Dimension 1', fontsize=12)
plt.ylabel('UMAP Dimension 2', fontsize=12)
plt.colorbar(label='Gender-Education Combination')
plt.show()


# Modeling

The implementation of logistic regression must be written only from the examples given to you by the instructor. No credit will be assigned to teams that copy implementations from another source, regardless of if the code is properly cited. 


In [14]:
# Implementation of logistic regression according to instructor course
# Reference: https://github.com/eclarson/MachineLearningNotebooks/blob/master/05.%20Logistic%20Regression.ipynb

import numpy as np
class BinaryLogisticRegressionBase:
    # private:
    def __init__(self, eta, iterations=20):
        self.eta = eta
        self.iters = iterations
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        return 'Base Binary Logistic Regression Object, Not Trainable'
    
    # convenience, private and static:
    @staticmethod
    def _sigmoid(theta):
        return 1/(1+np.exp(-theta)) 
    
    @staticmethod
    def _add_intercept(X):
        return np.hstack((np.ones((X.shape[0],1)),X)) # add bias term
    
    # public:
    def predict_proba(self, X, add_intercept=True):
        # add bias term if requested
        Xb = self._add_intercept(X) if add_intercept else X
        return self._sigmoid(Xb @ self.w_) # return the probability y=1
    
    def predict(self,X):
        return (self.predict_proba(X)>0.5) #return the actual prediction
    
    
        
blr = BinaryLogisticRegressionBase(0.1)
print(blr)



Base Binary Logistic Regression Object, Not Trainable


### Create a custom, one-versus-all logistic regression classifier using numpy and scipy to optimize. Use object oriented conventions identical to scikit-learn. You should start with the template developed by the instructor in the course. You should add the following functionality to the logistic regression classifier:

- Ability to choose optimization technique when class is instantiated: either steepest ascent, stochastic gradient ascent, and Newton's method. It is recommended to call this the "solver" input for the class.

- Update the gradient calculation to include a customizable regularization term (either using no regularization, L1 regularization, L2 regularization, or both L1 and L2 regularization). Associate a cost with the regularization term, "C", that can be adjusted when the class is instantiated.  


###  Train your classifier to achieve good generalization performance. That is, adjust the optimization technique and the value of the regularization term(s) "C" to achieve the best performance on your test set. Visualize the performance of the classifier versus the parameters you investigated.


Is your method of selecting parameters justified? That is, do you think there is any "data snooping" involved with this method of selecting parameters?

### [1.5 points] Compare the performance of your "best" logistic regression optimization procedure to the procedure used in scikit-learn. Visualize the performance differences in terms of training time and classification performance. Discuss the results. 


# Deployment

Which implementation of logistic regression would you advise be used in a deployed machine learning model, your implementation or scikit-learn (or other third party implementation)? Why?


# Exceptional Work (Required for 7000)


Implement an optimization technique for logistic regression using mean square error as your objective function (instead of maximum likelihood). Derive the gradient updates for the Hessian and use Newton's method to update the values of "w". Then answer, which process do you prefer: maximum likelihood OR minimum mean-squared error?


